In [122]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [123]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [124]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [125]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Importing**

In [126]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

url = "https://morepower.com.ph/monthly-rates/"
pdf_folder = "pdf downloads"

# Create the folder if it doesn't exist
os.makedirs(pdf_folder, exist_ok=True)

# Define headers to include in the request
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send a GET request to the webpage with headers
response = requests.get(url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all links that end with .pdf
    pdf_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.pdf')]
    
    # Download each PDF
    for link in pdf_links:
        pdf_url = urljoin(url, link)  # Create the full URL if it's relative
        pdf_filename = os.path.join(pdf_folder, os.path.basename(pdf_url))
        
        # Send a GET request to the PDF URL
        pdf_response = requests.get(pdf_url, headers=headers)
        
        if pdf_response.status_code == 200:
            # Save the PDF content
            with open(pdf_filename, "wb") as pdf_file:
                pdf_file.write(pdf_response.content)
            print(f"Downloaded {pdf_filename}")
        else:
            print(f"Failed to download {pdf_url}. Status code: {pdf_response.status_code}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Downloaded pdf downloads/Generation-Charge_July2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_June-2024_Web-Upload_revised0618.pdf
Downloaded pdf downloads/Generation-Charge_May-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Apr-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Mar-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Feb-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Jan-2024_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Dec-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Nov-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Oct-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_Sep-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_August-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_July-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge_June-2023_Web-Upload.pdf
Downloaded pdf downloads/Generation-Charge-f

### **Processing**

In [286]:
# Initialize dictionaries to store keys for DataFrames with and without "Supplier"
supplier_present_keys = {}
supplier_absent_keys = {}

# Loop through all PDF files in the folder
for filename in os.listdir(pdf_folder_path):
    try:
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder_path, filename)

            # Read all tables from the specified page(s)
            tables = tabula.read_pdf(pdf_path, lattice=True, pages='all', multiple_tables=True)

            # Ensure there is at least one table
            if not tables:
                raise ValueError(f"No tables found in the PDF: {filename}")

            # Process the first table (assuming it has headers)
            first_table = tables[0]

            # Extract column names from the first table
            first_table_columns = first_table.columns

            # Initialize a list to store processed DataFrames
            processed_tables = [first_table]

            # Process each subsequent table
            for table in tables[1:]:
                # Convert the header row of the current table to a DataFrame row
                table_header_as_row = pd.DataFrame([table.columns.tolist()], columns=table.columns)
                
                # Append the header row to the current table data
                table_with_header_as_row = pd.concat([table_header_as_row, table], ignore_index=True)

                # Check if the number of columns matches
                if len(table_with_header_as_row.columns) != len(first_table_columns):
                    # Align columns by adding missing columns with NaN values
                    missing_cols = len(first_table_columns) - len(table_with_header_as_row.columns)
                    if missing_cols > 0:
                        # Add missing columns with NaN values
                        table_with_header_as_row = pd.concat(
                            [table_with_header_as_row, pd.DataFrame(columns=[f"Column_{i+1}" for i in range(missing_cols)])], 
                            axis=1
                        )
                    elif missing_cols < 0:
                        # Truncate extra columns
                        table_with_header_as_row = table_with_header_as_row.iloc[:, :len(first_table_columns)]
                
                # Rename columns of the current table to match the first table
                table_with_header_as_row.columns = first_table_columns
                
                # Append the table to the list of processed tables
                processed_tables.append(table_with_header_as_row)

            # Combine all tables into a single DataFrame, stacking vertically
            df = pd.concat(processed_tables, ignore_index=True)

            # Check if any of the column headers contain the word "Supplier"
            if any('Supplier' in col or 'Power' in col for col in df.columns):
                # Store the key in supplier_present_keys dictionary
                supplier_present_keys[filename] = df
            else:
                # Store the key in supplier_absent_keys dictionary
                supplier_absent_keys[filename] = df

    except Exception as e:
        print(f"Error processing file {filename}: {e}")

# Output the keys for verification
print("Files with 'Supplier' in headers:", supplier_present_keys.keys())
print("Files without 'Supplier' in headers:", supplier_absent_keys.keys())

Files with 'Supplier' in headers: dict_keys(['Generation-Charge_June-2024_Web-Upload_revised0618.pdf', 'Generation-Charge_July2024_Web-Upload.pdf', 'Generation-Charge_Jan-2024_Web-Upload.pdf', 'Generation-Charge-for-June-2022.pdf', 'Generation-Charge-for-July-2022.pdf', 'Generation-Charge-for-November-2022.pdf', 'Generation-Charge_Sep-2023_Web-Upload.pdf', 'Generation-Charge_Oct-2023_Web-Upload.pdf', 'Generation-Charge-for-November-2021.pdf', 'Generation-Charge_Dec-2023_Web-Upload.pdf', 'Generation-Charge-for-March-2023.pdf', 'Generation-Charge_Nov-2023_Web-Upload.pdf', 'Generation-Charge-for-March-2022.pdf', 'Generation-Charge-for-April-2023.pdf', 'Generation-Charge-for-April-2022.pdf', 'Generation-Charge_Feb-2024_Web-Upload.pdf', 'Generation-Charge_June-2023_Web-Upload.pdf', 'Generation-Charge_Mar-2024_Web-Upload.pdf', 'Generation-Charge-for-September-2021.pdf', 'Generation-Charge-for-February-2023.pdf', 'Generation-Charge-for-February-2022.pdf', 'Generation-Charge-for-May-2022.pdf',

#### for dataframes w/ column headers

In [287]:
supplier_present_keys.keys()

dict_keys(['Generation-Charge_June-2024_Web-Upload_revised0618.pdf', 'Generation-Charge_July2024_Web-Upload.pdf', 'Generation-Charge_Jan-2024_Web-Upload.pdf', 'Generation-Charge-for-June-2022.pdf', 'Generation-Charge-for-July-2022.pdf', 'Generation-Charge-for-November-2022.pdf', 'Generation-Charge_Sep-2023_Web-Upload.pdf', 'Generation-Charge_Oct-2023_Web-Upload.pdf', 'Generation-Charge-for-November-2021.pdf', 'Generation-Charge_Dec-2023_Web-Upload.pdf', 'Generation-Charge-for-March-2023.pdf', 'Generation-Charge_Nov-2023_Web-Upload.pdf', 'Generation-Charge-for-March-2022.pdf', 'Generation-Charge-for-April-2023.pdf', 'Generation-Charge-for-April-2022.pdf', 'Generation-Charge_Feb-2024_Web-Upload.pdf', 'Generation-Charge_June-2023_Web-Upload.pdf', 'Generation-Charge_Mar-2024_Web-Upload.pdf', 'Generation-Charge-for-September-2021.pdf', 'Generation-Charge-for-February-2023.pdf', 'Generation-Charge-for-February-2022.pdf', 'Generation-Charge-for-May-2022.pdf', 'Generation-Charge-for-August-202

In [288]:
dataframes = []

from datetime import datetime

def extract_date_key(filename):
    # Define patterns for different filename formats
    patterns = [
        r'(\w+)-(\d{4})',        # Matches "Month-YYYY" (e.g., "June-2024")
        r'(\d{4})-(\d{2})',      # Matches "YYYY-MM" (e.g., "2024-06")
        r'(\d{2})-(\d{2})',      # Matches "MM-YY" (e.g., "06-24")
        r'(\d{4})(\d{2})',       # Matches "YYYYMM" (e.g., "202406")
        r'(\d{2})-(\d{4})',      # Matches "MM-YYYY" (e.g., "06-2024")
        r'(\d{4})-(\w+)',        # Matches "YYYY-Month" (e.g., "2024-July")
        r'(\w+)(\d{4})'          # Matches "MonthYYYY" (e.g., "July2024")
    ]
    
    # Define month abbreviation to number mapping
    month_abbr_to_num = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
        'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12',
        'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05',
        'June': '06', 'July': '07', 'August': '08', 'September': '09', 'October': '10',
        'November': '11', 'December': '12'
    }
    
    # Extract text after the first underscore and remove file extension
    parts = filename.split('_', 1)
    filename = parts[1] if len(parts) > 1 else filename
    filename = re.sub(r'\.pdf$', '', filename)
    
    # Try to find a match with each pattern
    for pattern in patterns:
        match = re.search(pattern, filename)
        if match:
            groups = match.groups()
            if len(groups) == 2:
                month, year = groups
                if month.isdigit():
                    # Format: YYYY-MM, MM-YY, or YYYYMM
                    if len(month) == 2:  # MM or YY
                        return f"{month}-{year[-2:]}"  # MM-YY format
                    else:  # YYYYMM
                        return f"{month[-2:]}-{year[-2:]}"  # MM-YY format
                elif month in month_abbr_to_num:
                    # Handle formats with Month-YYYY or YYYY-Month
                    month_num = month_abbr_to_num[month]
                    return f"{month_num}-{year[-2:]}"  # MM-YY format
            elif len(groups) == 3:
                month, year = groups[1], groups[2]
                # Handle formats with Month-YYMM
                if month in month_abbr_to_num:
                    month_num = month_abbr_to_num[month]
                    return f"{month_num}-{year[-2:]}"  # MM-YY format
            elif len(groups) == 1 and groups[0] in month_abbr_to_num:
                # Handle formats with MonthYYYY
                month_num = month_abbr_to_num[groups[0]]
                return f"{month_num}-{year[-2:]}"  # MM-YY format

    # If no pattern matches, return an error
    raise ValueError(f"Date not found in filename: {filename}")

# Process each file in the dictionary
for filename, df in supplier_present_keys.items():
    # Slicing columns
    keywords = ["Power", "%", "Purchased", "Total"]
    filtered_columns = [col for col in df.columns if any(keyword in col for keyword in keywords)]
    sliced_columns = df[filtered_columns]

    # Slicing rows
    first_non_null_idx = sliced_columns[sliced_columns.iloc[:, 1].notna()].index[0]
    wesm_row_idx = sliced_columns[sliced_columns.iloc[:, 0].str.contains("WESM", na=False, case=False)].index
    total_row_idx = sliced_columns[sliced_columns.iloc[:, 0].str.contains("TOTAL", na=False, case=False)].index

    if not wesm_row_idx.empty and not total_row_idx.empty:
        wesm_row_idx = wesm_row_idx[0]
        total_row_idx = total_row_idx[0]

        sliced_rows = sliced_columns.loc[first_non_null_idx:wesm_row_idx]
        
        if total_row_idx > wesm_row_idx:
            total_row = sliced_columns.loc[[total_row_idx]]
            sliced_rows = pd.concat([sliced_rows, total_row], ignore_index=True)

    # Clean the "Power Supplier" column
    def clean_power_supplier(value):
        if pd.isnull(value):
            return value
        return re.sub(r'^\d+\.\s*', '', value)
    
    sliced_rows.iloc[:, 0] = sliced_rows.iloc[:, 0].apply(clean_power_supplier)
    sliced_rows = sliced_rows.dropna(subset=[sliced_rows.columns[0]])
    sliced_rows = sliced_rows.reset_index(drop=True)

    # Calculate the Average Generation Cost
    def calculate_average_generation_cost(row):
        try:
            total_cost = float(row['Total Generation Cost'].replace(',', ''))
            kwh = float(row['kWh'].replace(',', ''))
            return total_cost / kwh
        except (ValueError, TypeError):
            return np.nan

    if 'Total Generation Cost' in sliced_rows.columns and 'kWh' in sliced_rows.columns:
        sliced_rows['Average Generation Cost'] = sliced_rows.apply(calculate_average_generation_cost, axis=1)
        sliced_rows = sliced_rows.drop(columns=['Total Generation Cost'])

    # Rename the columns
    sliced_rows.columns = ["Power Supplier", "%", "kWh", "Average Generation Cost"]

    # Add the Date column
    date_str = extract_date_key(filename)
    if date_str:
        sliced_rows['Date'] = date_str

    # Append the DataFrame to the list
    dataframes.append(sliced_rows)

# Concatenate all DataFrames into one
big_df = pd.concat(dataframes, ignore_index=True)

big_df.head()

,Power Supplier,%,kWh,Average Generation Cost,Date
0,Sem Calaca Power Corp.,25.84%,"17,825,000.00","82,160,418.62",06-24
1,KEPCO SPC Power Corp.,20.58%,"14,200,000.00","69,175,290.48",06-24
2,Energy Development Corp,14.05%,"9,692,000.00","61,514,203.44",06-24
3,Spot Market (WESM),39.30%,"27,109,551.24","61,868,333.90",06-24
4,SUBTOTAL for Power Suppliers,100.00%,"68,988,059.24","275,737,392.40",06-24


#### for dataframes w/o column headers

In [289]:
supplier_absent_keys.keys()

dict_keys(['GenCharge_Mar2020.pdf', 'GenCharge_Jul2020.pdf', 'GenCharge_May2020.pdf', 'GenCharge_Aug2020.pdf', 'GenCharge_Oct2020.pdf', 'GenCharge_Sep2020.pdf', 'GenCharge_Apr2020.pdf', 'GenCharge_Dec2020.pdf', 'GenCharge_Nov2020.pdf'])

In [290]:
# Iterate over each item in the dictionary
for key, df in supplier_absent_keys.items():
    # Define the keywords to search for
    keywords = ["CONTRACTS", "%", "Purchased", "Average"]

    # Create a list to store the column indices that match the criteria
    matching_columns = []

    # Find columns where the keywords are present as row values
    for keyword in keywords:
        if keyword == "Purchased":
            # For "Purchased", we need to handle the second occurrence
            purchased_columns = df.apply(lambda col: col.str.contains(keyword, na=False)).any()
            # Get the indices of all columns with "Purchased"
            purchased_indices = purchased_columns[purchased_columns].index.tolist()
            if len(purchased_indices) > 1:
                # Select the second occurrence
                matching_columns.append(purchased_indices[1])
        else:
            # For all other keywords, find columns containing the keyword as a row value
            matching_columns += df.columns[df.apply(lambda col: col.str.contains(keyword, na=False)).any()].tolist()

    # Drop duplicates in case there are overlaps in columns
    matching_columns = pd.Index(matching_columns).unique()

    # Create the new DataFrame with the selected columns
    sliced_columns = df[matching_columns]

    # Attempt to find the index of the row containing the word "CONTRACTS" in the first column
    contract_indices = sliced_columns[sliced_columns.iloc[:, 0].str.contains('CONTRACTS', case=False, na=False)].index

    if not contract_indices.empty:
        # If "CONTRACTS" is found, use it as the start index
        start_index = contract_indices[0] + 1
    else:
        # If "CONTRACTS" is not found, find the first non-null value in the first column
        first_non_null_index = sliced_columns[sliced_columns.iloc[:, 0].notnull()].index[0]
        start_index = first_non_null_index  # Include this row as well

    # Find the index of the row containing the word "WESM" in the first column
    wesm_index = sliced_columns[sliced_columns.iloc[:, 0].str.contains(r'WESM', case=False, na=False)].index[0]

    # Slice the DataFrame to include rows between start_index and end_index (inclusive)
    sliced_rows = sliced_columns.iloc[start_index:wesm_index + 1]

    # Exclude the row that contains the word "Metering" in the first column
    sliced_rows = sliced_rows[~sliced_rows.iloc[:, 0].str.contains('Metering', case=False, na=False)]

    # Uncomment and adjust the following line if you need to rename columns
    sliced_rows.columns = ["Power Supplier", "%", "kWh", "Average Generation Cost"]

    # Function to clean the "Power Supplier" column
    def clean_power_supplier(value):
        # Remove leading numbers and periods
        value = re.sub(r'^\d+\.\s*', '', value)
        return value

    # Apply the cleaning function to the first column of sliced_rows
    sliced_rows['Power Supplier'] = sliced_rows['Power Supplier'].apply(clean_power_supplier)

    # Find the column that contains the word "Average"
    average_column = sliced_columns.columns[sliced_columns.apply(lambda col: col.str.contains('Average', case=False, na=False)).any()][0]

    # Find rows where the first column contains the word "TOTAL"
    total_row = sliced_columns.loc[sliced_columns.iloc[:, 0].str.contains("TOTAL", case=False, na=False), average_column]
    reversed_row = total_row[::-1]

    for value in reversed_row:
        if pd.notnull(value):
            total_row_value = value
            break  # Exit loop once a non-null value is found

    # Add the "Generation Charge" to the sliced_rows DataFrame
    sliced_rows["Generation Charge"] = total_row_value

    # Add the date column
    date_str = extract_date_key(key)
    sliced_rows['Date'] = date_str

    # Append to big_df
    big_df = pd.concat([big_df, sliced_rows], ignore_index=True)

big_df

,Power Supplier,%,kWh,Average Generation Cost,Date,Generation Charge
0,Sem Calaca Power Corp.,25.84%,"17,825,000.00","82,160,418.62",06-24,NaN
1,KEPCO SPC Power Corp.,20.58%,"14,200,000.00","69,175,290.48",06-24,NaN
2,Energy Development Corp,14.05%,"9,692,000.00","61,514,203.44",06-24,NaN
3,Spot Market (WESM),39.30%,"27,109,551.24","61,868,333.90",06-24,NaN
4,SUBTOTAL for Power Suppliers,100.00%,"68,988,059.24","275,737,392.40",06-24,NaN
5,Sem Calaca Power Corp.,29.98%,"18,125,000.00","87,651,359.32",07-24,NaN
6,KEPCO SPC Power Corp.,23.99%,"14,500,000.00","71,508,884.44",07-24,NaN
7,Energy Development Corp,16.01%,"9,678,000.00","59,823,084.60",07-24,NaN
8,Spot Market (WESM),NaN,NaN,-,07-24,NaN
9,SUBTOTAL for Power Suppliers,100.00%,"60,453,542.35","457,639,313.23",07-24,NaN


#### for troubleshooting loop

##### for dataframes w/ column headers

In [326]:
supplier_present_keys.keys()

dict_keys(['Generation-Charge_June-2024_Web-Upload_revised0618.pdf', 'Generation-Charge_July2024_Web-Upload.pdf', 'Generation-Charge_Jan-2024_Web-Upload.pdf', 'Generation-Charge-for-June-2022.pdf', 'Generation-Charge-for-July-2022.pdf', 'Generation-Charge-for-November-2022.pdf', 'Generation-Charge_Sep-2023_Web-Upload.pdf', 'Generation-Charge_Oct-2023_Web-Upload.pdf', 'Generation-Charge-for-November-2021.pdf', 'Generation-Charge_Dec-2023_Web-Upload.pdf', 'Generation-Charge-for-March-2023.pdf', 'Generation-Charge_Nov-2023_Web-Upload.pdf', 'Generation-Charge-for-March-2022.pdf', 'Generation-Charge-for-April-2023.pdf', 'Generation-Charge-for-April-2022.pdf', 'Generation-Charge_Feb-2024_Web-Upload.pdf', 'Generation-Charge_June-2023_Web-Upload.pdf', 'Generation-Charge_Mar-2024_Web-Upload.pdf', 'Generation-Charge-for-September-2021.pdf', 'Generation-Charge-for-February-2023.pdf', 'Generation-Charge-for-February-2022.pdf', 'Generation-Charge-for-May-2022.pdf', 'Generation-Charge-for-August-202

In [327]:
df = supplier_present_keys["Generation-Charge_June-2024_Web-Upload_revised0618.pdf"]
df

,Power\rSupplier,% Share on Energy\rPurchased,Energy\rPurchased\rkWh,Basic\rGeneration Cost\rPhP,Other Cost\rAdjustment\rPhP,Total\rGeneration Cost\rPhP
0,NaN,(A),(B),(C),(D = B+C),NaN
1,Sem Calaca Power Corp.,25.84%,"17,825,000.00","82,160,418.62",-,"82,160,418.62"
2,KEPCO SPC Power Corp.,20.58%,"14,200,000.00","69,085,633.34","89,657.14","69,175,290.48"
3,Energy Development Corp,14.05%,"9,692,000.00","59,882,207.60","1,631,995.84","61,514,203.44"
4,Spot Market (WESM),39.30%,"27,109,551.24","61,437,954.98","430,378.92","61,868,333.90"
5,Net Metering Export Energy,0.23%,"161,508.00","1,019,145.96",NaN,"1,019,145.96"
6,SUBTOTAL for Power Suppliers,100.00%,"68,988,059.24","273,585,360.50","2,152,031.90","275,737,392.40"
7,Others:,NaN,NaN,NaN,NaN,NaN
8,Pilferage Cost Recoveries,NaN,NaN,NaN,NaN,"(935,260.84)"
9,Prompt Payment Discounts,NaN,NaN,NaN,NaN,NaN


In [328]:
#Slicing columns

# Define the keywords to look for in the column names
keywords = ["Power", "%", "Purchased", "Total"]

# Filter the columns based on the presence of any of the keywords
filtered_columns = [col for col in df.columns if any(keyword in col for keyword in keywords)]

# Create the new DataFrame with the filtered columns
sliced_columns = df[filtered_columns]

sliced_columns.columns = ["Power Supplier", "%", "kWh", "Total Generation Cost"]
sliced_columns

,Power Supplier,%,kWh,Total Generation Cost
0,NaN,(A),(B),NaN
1,Sem Calaca Power Corp.,25.84%,"17,825,000.00","82,160,418.62"
2,KEPCO SPC Power Corp.,20.58%,"14,200,000.00","69,175,290.48"
3,Energy Development Corp,14.05%,"9,692,000.00","61,514,203.44"
4,Spot Market (WESM),39.30%,"27,109,551.24","61,868,333.90"
5,Net Metering Export Energy,0.23%,"161,508.00","1,019,145.96"
6,SUBTOTAL for Power Suppliers,100.00%,"68,988,059.24","275,737,392.40"
7,Others:,NaN,NaN,NaN
8,Pilferage Cost Recoveries,NaN,NaN,"(935,260.84)"
9,Prompt Payment Discounts,NaN,NaN,NaN


In [333]:
#Slicing rows

# Step 1: Find the first non-null row index in the first column
first_non_null_idx = sliced_columns[sliced_columns.iloc[:, 1].notna()].index[0]

# Step 2: Find the row index where the value in the first column is "WESM"
wesm_row_idx = sliced_columns[sliced_columns.iloc[:, 0].str.contains("WESM", na=False, case=False)].index

# Step 3: Find the row index where the value in the first column is "TOTAL"
total_row_idx = sliced_columns[sliced_columns.iloc[:, 0].str.strip().eq("TOTAL")].index

# Step 4: Slice the DataFrame to include the first non-null row, up to "WESM", and include "TOTAL"
if not wesm_row_idx.empty and not total_row_idx.empty:
    wesm_row_idx = wesm_row_idx[0]  # Get the index of the "WESM" row
    total_row_idx = total_row_idx[0]  # Get the index of the "TOTAL" row
    
    # Slice rows from the first non-null row to the "WESM" row (inclusive)
    sliced_rows = sliced_columns.loc[first_non_null_idx:wesm_row_idx]
    
    # Append the "TOTAL" row if it's not already in the slice
    if total_row_idx > wesm_row_idx:
        total_row = sliced_columns.loc[[total_row_idx]]
        sliced_rows = pd.concat([sliced_rows, total_row], ignore_index=True)

sliced_rows.columns = ["Power Supplier", "%", "kWh", "Total Generation Cost"]
sliced_rows

,Power Supplier,%,kWh,Total Generation Cost
0,NaN,(A),(B),NaN
1,Sem Calaca Power Corp.,25.84%,"17,825,000.00","82,160,418.62"
2,KEPCO SPC Power Corp.,20.58%,"14,200,000.00","69,175,290.48"
3,Energy Development Corp,14.05%,"9,692,000.00","61,514,203.44"
4,Spot Market (WESM),39.30%,"27,109,551.24","61,868,333.90"
5,TOTAL,NaN,"68,988,059.24","274,802,131.56"


In [334]:
import re

# Function to clean the "Power Supplier" column
def clean_power_supplier(value):
    if pd.isnull(value):
        return value  # Return null if the value is null
    # Remove leading numbers and periods
    value = re.sub(r'^\d+\.\s*', '', value)
    return value

# Apply the cleaning function to the first column of sliced_rows
sliced_rows.iloc[:, 0] = sliced_rows.iloc[:, 0].apply(clean_power_supplier)

# Drop rows where the first column value is null
sliced_rows = sliced_rows.dropna(subset=[sliced_rows.columns[0]])

# Reset the index after dropping rows
sliced_rows = sliced_rows.reset_index(drop=True)

sliced_rows

,Power Supplier,%,kWh,Total Generation Cost
0,Sem Calaca Power Corp.,25.84%,"17,825,000.00","82,160,418.62"
1,KEPCO SPC Power Corp.,20.58%,"14,200,000.00","69,175,290.48"
2,Energy Development Corp,14.05%,"9,692,000.00","61,514,203.44"
3,Spot Market (WESM),39.30%,"27,109,551.24","61,868,333.90"
4,TOTAL,NaN,"68,988,059.24","274,802,131.56"


In [335]:
# Find the row where the value in the first column is "TOTAL"
total_row = sliced_columns[sliced_columns.iloc[:, 0].str.contains("TOTAL", na=False)]

# Extract the values for "Total Generation Cost" and "kWh"
if not total_row.empty:
    total_generation_cost_str = total_row['Total Generation Cost'].values[0]
    kwh_purchased_str = total_row['kWh'].values[0]

    # Convert the string values to float by removing commas
    total_generation_cost = float(total_generation_cost_str.replace(',', ''))
    kwh_purchased = float(kwh_purchased_str.replace(',', ''))

    # Calculate the Average Generation Cost for each row
    def calculate_average_generation_cost(row):
        try:
            total_cost = float(row['Total Generation Cost'].replace(',', ''))
            kwh = float(row['kWh'].replace(',', ''))
            return total_cost / kwh
        except (ValueError, TypeError):
            return np.nan

    sliced_rows['Average Generation Cost'] = sliced_rows.apply(calculate_average_generation_cost, axis=1)

    # Drop the "Total Generation Cost" column
    sliced_rows = sliced_rows.drop(columns=['Total Generation Cost'])

sliced_rows

,Power Supplier,%,kWh,Average Generation Cost
0,Sem Calaca Power Corp.,25.84%,"17,825,000.00",4.609280
1,KEPCO SPC Power Corp.,20.58%,"14,200,000.00",4.871499
2,Energy Development Corp,14.05%,"9,692,000.00",6.346905
3,Spot Market (WESM),39.30%,"27,109,551.24",2.282160
4,TOTAL,NaN,"68,988,059.24",3.983329


In [342]:
# Step 1: Find the row index where the value in the first column is exactly "TOTAL"
total_row_idx = sliced_rows[sliced_rows.iloc[:, 0] == "TOTAL"].index[0]

# Step 2: Extract the value from the "Average Generation Cost" column in the "TOTAL" row
generation_charge_value = sliced_rows.at[total_row_idx, "Average Generation Cost"]

# Step 3: Add a new column "Generation Charge" with this value repeated for all rows
sliced_rows["Generation Charge"] = generation_charge_value

# Step 4: Remove the "TOTAL" row
sliced_rows = sliced_rows.drop(total_row_idx)

sliced_rows

,Power Supplier,%,kWh,Average Generation Cost,Generation Charge
0,Sem Calaca Power Corp.,25.84%,"17,825,000.00",4.609280,3.983329
1,KEPCO SPC Power Corp.,20.58%,"14,200,000.00",4.871499,3.983329
2,Energy Development Corp,14.05%,"9,692,000.00",6.346905,3.983329
3,Spot Market (WESM),39.30%,"27,109,551.24",2.282160,3.983329


##### for dataframes w/o column headers

In [338]:
df = supplier_absent_keys["GenCharge_202101.pdf"]
df

KeyError: 'GenCharge_202101.pdf'

In [292]:
# Define the keywords to search for
keywords = ["CONTRACTS", "%", "Purchased", "Average"]

# Create a list to store the column indices that match the criteria
matching_columns = []

# Find columns where the keywords are present as row values
for keyword in keywords:
    if keyword == "Purchased":
        # For "Purchased", we need to handle the second occurrence
        purchased_columns = df.apply(lambda col: col.str.contains(keyword, na=False)).any()
        # Get the indices of all columns with "Purchased"
        purchased_indices = purchased_columns[purchased_columns].index.tolist()
        if len(purchased_indices) > 1:
            # Select the second occurrence
            matching_columns.append(purchased_indices[1])
    else:
        # For all other keywords, find columns containing the keyword as a row value
        matching_columns += df.columns[df.apply(lambda col: col.str.contains(keyword, na=False)).any()].tolist()

# Drop duplicates in case there are overlaps in columns
matching_columns = pd.Index(matching_columns).unique()

# Create the new DataFrame with the selected columns
sliced_columns = df[matching_columns]

sliced_columns

,GENERATION CHARGE for NOVEMBER 2020,Unnamed: 0,Unnamed: 1,Unnamed: 5
0,Power Supplier,% Share on\rEnergy\rPurchased,Energy\rPurchased\rkWh,Average\rGeneration Rate\rPhP/kWh
1,NaN,(A),(B),NaN
2,BILATERAL CONTRACTS,NaN,NaN,NaN
3,1. Panay Energy Development Corporation U1/U2,41.32%,"19,722,394.09",6.7546
4,2. Panay Energy Development Corporation U3,29.84%,"14,244,166.58",6.0243
5,3. Panay Power Corporation,0.23%,"111,676.64",227.9351
6,4. KEPCO SPC Power Corporation,7.54%,"3,600,000.00",4.6738
7,"5. Aboitiz Power Renewables, Incorporated",6.61%,"3,155,000.00",6.7343
8,Subtotal - Emergency Power Supply Contracts,85.55%,"40,833,237.31",6.9197
9,Wholesale Electricity Spot Market (WESM),14.40%,"6,875,306.91",2.1797


In [293]:
# Attempt to find the index of the row containing the word "CONTRACTS" in the first column
contract_indices = sliced_columns[sliced_columns.iloc[:, 0].str.contains('CONTRACTS', case=False, na=False)].index

if not contract_indices.empty:
    # If "CONTRACTS" is found, use it as the start index
    start_index = contract_indices[0] + 1
else:
    # If "CONTRACTS" is not found, find the first non-null value in the first column
    first_non_null_index = sliced_columns[sliced_columns.iloc[:, 0].notnull()].index[0]
    start_index = first_non_null_index  # Include this row as well

# Find the index of the row containing the word "WESM" in the first column
wesm_index = sliced_columns[sliced_columns.iloc[:, 0].str.contains(r'WESM', case=False, na=False)].index[0]

# Slice the DataFrame to include rows between start_index and end_index (inclusive)
sliced_rows = sliced_columns.iloc[start_index:end_index + 1]

# Exclude the row that contains the word "Metering" in the first column
sliced_rows = sliced_rows[~sliced_rows.iloc[:, 0].str.contains('Metering', case=False, na=False)]

# Uncomment and adjust the following line if you need to rename columns
sliced_rows.columns = ["Power Supplier", "%", "kWh", "Average Generation Cost"]

sliced_rows

,Power Supplier,%,kWh,Average Generation Cost
3,1. Panay Energy Development Corporation U1/U2,41.32%,"19,722,394.09",6.7546
4,2. Panay Energy Development Corporation U3,29.84%,"14,244,166.58",6.0243
5,3. Panay Power Corporation,0.23%,"111,676.64",227.9351
6,4. KEPCO SPC Power Corporation,7.54%,"3,600,000.00",4.6738
7,"5. Aboitiz Power Renewables, Incorporated",6.61%,"3,155,000.00",6.7343
8,Subtotal - Emergency Power Supply Contracts,85.55%,"40,833,237.31",6.9197


In [294]:
import re

# Function to clean the "Power Supplier" column
def clean_power_supplier(value):
    # Remove leading numbers and periods
    value = re.sub(r'^\d+\.\s*', '', value)
    return value

# Apply the cleaning function to the first column of sliced_df
sliced_rows.iloc[:, 0] = sliced_rows.iloc[:, 0].apply(clean_power_supplier)

sliced_rows_df = sliced_rows.reset_index(drop=True)
sliced_rows_df

,Power Supplier,%,kWh,Average Generation Cost
0,Panay Energy Development Corporation U1/U2,41.32%,"19,722,394.09",6.7546
1,Panay Energy Development Corporation U3,29.84%,"14,244,166.58",6.0243
2,Panay Power Corporation,0.23%,"111,676.64",227.9351
3,KEPCO SPC Power Corporation,7.54%,"3,600,000.00",4.6738
4,"Aboitiz Power Renewables, Incorporated",6.61%,"3,155,000.00",6.7343
5,Subtotal - Emergency Power Supply Contracts,85.55%,"40,833,237.31",6.9197


In [296]:
# Find rows where the first column contains the word "TOTAL"
total_row = sliced_columns.loc[sliced_columns.iloc[:, 0].str.contains("TOTAL", case=False, na=False), average_column]
reversed_row = total_row[::-1]

for value in reversed_row:
    if pd.notnull(value):
        total_row_value = value
        break  # Exit loop once a non-null value is found

In [297]:
sliced_rows_df["Generation Charge"] = total_row_value
sliced_rows_df

,Power Supplier,%,kWh,Average Generation Cost,Generation Charge
0,Panay Energy Development Corporation U1/U2,41.32%,"19,722,394.09",6.7546,6.2367
1,Panay Energy Development Corporation U3,29.84%,"14,244,166.58",6.0243,6.2367
2,Panay Power Corporation,0.23%,"111,676.64",227.9351,6.2367
3,KEPCO SPC Power Corporation,7.54%,"3,600,000.00",4.6738,6.2367
4,"Aboitiz Power Renewables, Incorporated",6.61%,"3,155,000.00",6.7343,6.2367
5,Subtotal - Emergency Power Supply Contracts,85.55%,"40,833,237.31",6.9197,6.2367
